This notebook is associated with Section 3 (Algebras) of the paper ["A fundamental non-classical logic"](https://arxiv.org/abs/2207.06993) by [Wesley H. Holliday](mailto:wesholliday@berkeley.edu).

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s [interface](https://www.nltk.org/howto/inference.html) to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to verify basic facts about the kinds of negations discussed in Section 3 of the paper.

To view the notebook online, type the URL of this notebook (https://github.com/wesholliday/fundamental-logic/blob/main/negation-axioms.ipynb) into the location field at https://nbviewer.org. GitHub's preview of the notebook might not show all the output that was generated.

## Outline

**1. [Lattice axioms](#1)**

**2. [Bounded lattice axioms](#2)**

**3. [Negation axioms](#3)**

**4. [Lemma 3.2](#4)**

**5. [Figure 6](#5)**

**6. [Any bounded lattice admits a weak pseudocomplementation](#6)**

**7. [Any lattice $L$ with $|L|>2$ admits a precomplementation that is not a protocomplementation](#7)**

In [101]:
from nltk.test.inference_fixt import setup_module 

setup_module()

from nltk import *
from nltk.sem.drt import DrtParser
from nltk.sem import logic
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## 1. Lattice axioms<a id='1'></a>

In [102]:
or_id = read_expr('Or(x,x) = x')
and_id = read_expr('And(x,x) = x')

or_comm = read_expr('Or(x,y) = Or(y,x)')
and_comm = read_expr('And(x,y) = And(y,x)')

or_assoc = read_expr('Or(x,Or(y,z)) = Or(Or(x,y),z)')
and_assoc = read_expr('And(x,And(y,z)) = And(And(x,y),z)')

or_absorp = read_expr('Or(x,And(x,y)) = x')
and_absorp = read_expr('And(x,Or(x,y)) = x')

lattice = [or_id, and_id, or_comm, and_comm, or_assoc, and_assoc, or_absorp, and_absorp]

In [103]:
#By including a definition of the covering relation,
#one can quickly read off Hasse diagrams of lattices from the Mace4 output.
cover_def = read_expr('covered_by(x,y) <-> (-(x=y) & x = And(x,y) & -exists z.(-(z=x) & -(z=y) & x = And(x,z) & z = And(z,y)))')

## 2. Bounded lattice axioms<a id='2'></a>

In [104]:
bot = read_expr('Or(x,Bot) = x')
top = read_expr('And(x,Top) = x')

bounded_lattice = lattice + [bot,top]

## 3. Negation axioms<a id='3'></a>

In [105]:
antitone = read_expr('x=And(x,y) -> Not(y)=And(Not(x),Not(y))')

top_to_bot = read_expr('Not(Top)=Bot')

bot_to_top = read_expr('Not(Bot)=Top')

semicomp = read_expr('And(x,Not(x)) = Bot')

DNI = read_expr('x=And(x,Not(Not(x)))')

bot_to_neg = read_expr('And(x,y) = Bot -> x=And(x,Not(y))')

DNE = read_expr('Not(Not(x))=And(Not(Not(x)),x)')

LEM = read_expr('Or(x,Not(x)) = Top')

In [106]:
pseudo = bounded_lattice + [bot_to_neg, semicomp]

ortho = bounded_lattice + [antitone, semicomp, LEM, DNI, DNE]

protocomp = bounded_lattice + [antitone, semicomp, bot_to_top]

precomp = bounded_lattice + [antitone, top_to_bot]

weak_pseudo = bounded_lattice + [antitone, semicomp, DNI]

ultraweak_pseudo = bounded_lattice + [antitone, DNI, top_to_bot]

In [107]:
anti_inflationary = read_expr('-(x=Bot) -> -(x=And(x,Not(x)))')

weak_pseudo_ax = read_expr('x=And(x,Not(y)) -> y=And(y,Not(x))')

# 4. Lemma 3.2<a id='4'></a>

## Lemma 3.2.1

In [108]:
goal = anti_inflationary
prover = Prover9Command(goal, assumptions = [semicomp])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18866 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 7.
% Level of proof is 3.
% Maximum clause weight is 6.000.
% Given clauses 0.


1 x != Bot -> x != And(x,Not(x)).  [goal].
2 And(x,Not(x)) = Bot.  [assumption].
3 Bot != c1.  [deny(1)].
4 c1 != Bot.  [copy(3),flip(a)].
5 And(c1,Not(c1)) = c1.  [deny(1)].
6 c1 = Bot.  [copy(5),rewrite([2(4)]),flip(a)].
7 $F.  [resolve(6,a,4,a)].

============================== end of proof ==========================


In [109]:
goal = semicomp
prover = Prover9Command(goal, assumptions = bounded_lattice + [antitone,anti_inflationary])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18868 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 16.
% Level of proof is 5.
% Maximum clause weight is 15.000.
% Given clauses 75.


1 x = And(x,y) -> Not(y) = And(Not(x),Not(y)).  [assumption].
2 x != Bot -> x != And(x,Not(x)).  [assumption].
3 And(x,Not(x)) = Bot.  [goal].
5 And(x,x) = x.  [assumption].
7 And(x,y) = And(y,x).  [assumption].
10 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
11 And(And(x,y),z) = And(x,And(y,z)).  [copy(10),flip(a)].
16 And(x,y) != x | Not(y) 

## Lemma 3.2.2

In [110]:
goal = weak_pseudo_ax
prover = Prover9Command(goal, assumptions = bounded_lattice + [antitone,DNI])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18870 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 23.
% Level of proof is 6.
% Maximum clause weight is 13.000.
% Given clauses 57.


1 x = And(x,y) -> Not(y) = And(Not(x),Not(y)).  [assumption].
2 x = And(x,Not(y)) -> y = And(y,Not(x)).  [goal].
5 Or(x,y) = Or(y,x).  [assumption].
6 And(x,y) = And(y,x).  [assumption].
7 Or(x,Or(y,z)) = Or(Or(x,y),z).  [assumption].
8 Or(Or(x,y),z) = Or(x,Or(y,z)).  [copy(7),flip(a)].
11 Or(x,And(x,y)) = x.  [assumption].
12 And(x,Or(x,y)) = x.  

In [111]:
goal = antitone
prover = Prover9Command(goal, assumptions = bounded_lattice + [weak_pseudo_ax])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18872 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 15.
% Level of proof is 5.
% Maximum clause weight is 12.000.
% Given clauses 40.


1 x = And(x,Not(y)) -> y = And(y,Not(x)).  [assumption].
2 x = And(x,y) -> Not(y) = And(Not(x),Not(y)).  [goal].
4 And(x,x) = x.  [assumption].
6 And(x,y) = And(y,x).  [assumption].
9 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
10 And(And(x,y),z) = And(x,And(y,z)).  [copy(9),flip(a)].
15 And(x,Not(y)) != x | And(y,Not(x)) = y.  [clausify(1)].

In [112]:
goal = DNI
prover = Prover9Command(goal, assumptions = bounded_lattice + [weak_pseudo_ax])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18874 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 7.
% Level of proof is 3.
% Maximum clause weight is 12.000.
% Given clauses 11.


1 x = And(x,Not(y)) -> y = And(y,Not(x)).  [assumption].
2 x = And(x,Not(Not(x))).  [goal].
4 And(x,x) = x.  [assumption].
15 And(x,Not(y)) != x | And(y,Not(x)) = y.  [clausify(1)].
16 And(c1,Not(Not(c1))) != c1.  [deny(2)].
37 And(x,Not(Not(x))) = x.  [hyper(15,a,4,a)].
38 $F.  [resolve(37,a,16,a)].

============================== end of proof ====

## Lemma 3.2.3

In [113]:
goal = LEM
prover = Prover9Command(goal, assumptions = weak_pseudo + [DNE])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18876 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 33.
% Level of proof is 9.
% Maximum clause weight is 13.000.
% Given clauses 71.


1 x = And(x,y) -> Not(y) = And(Not(x),Not(y)).  [assumption].
2 Or(x,Not(x)) = Top.  [goal].
5 Or(x,y) = Or(y,x).  [assumption].
6 And(x,y) = And(y,x).  [assumption].
9 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
10 And(And(x,y),z) = And(x,And(y,z)).  [copy(9),flip(a)].
11 Or(x,And(x,y)) = x.  [assumption].
12 And(x,Or(x,y)) = x.  [assumption

# 5. Figure 6<a id='5'></a>

## Protocomplementation

In [114]:
goal = top_to_bot
prover = Prover9Command(goal, assumptions = protocomp)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18878 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 11.
% Level of proof is 4.
% Maximum clause weight is 13.000.
% Given clauses 11.


1 x = And(x,y) -> Not(y) = And(Not(x),Not(y)).  [assumption].
2 Not(Top) = Bot.  [goal].
15 And(x,y) != x | Not(y) = And(Not(x),Not(y)).  [clausify(1)].
16 And(x,y) != x | And(Not(x),Not(y)) = Not(y).  [copy(15),flip(b)].
17 And(x,Not(x)) = Bot.  [assumption].
18 Not(Bot) = Top.  [assumption].
19 Top = Not(Bot).  [copy(18),flip(a)].
20 Not(Top) != 

## Ultraweak pseudocomplementation

In [115]:
goal = bot_to_top
prover = Prover9Command(goal, assumptions = ultraweak_pseudo)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18880 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 12.
% Level of proof is 3.
% Maximum clause weight is 7.000.
% Given clauses 16.


2 Not(Bot) = Top.  [goal].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
17 x = And(x,Not(Not(x))).  [assumption].
18 And(x,Not(Not(x))) = x.  [copy(17),flip(a)].
19 Not(Top) = Bot.  [assumption].
20 Bot = Not(Top).  [copy(19),flip(a)].
21 Not(Bot) != Top.  [deny(2)].
22 Not(Not(Top)) != Top.  [copy(21),rewrite([20(1)])].
4

In [116]:
goal = semicomp
mb = MaceCommand(goal, assumptions = ultraweak_pseudo + [cover_def])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 3

Bot = 0.

Top = 1.

c1 = 2.

Not(0) = 1.
Not(1) = 0.
Not(2) = 2.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.

f1(0,0) = 0.
f1(0,1) = 2.
f1(0,2) = 0.
f1(1,0) = 0.
f1(1,1) = 0.
f1(1,2) = 0.
f1(2,0) = 0.
f1(2,1) = 0.
f1(2,2) = 0.

- covered_by(0,0).
- covered_by(0,1).
  covered_by(0,2).
- covered_by(1,0).
- covered_by(1,1).
- covered_by(1,2).
- covered_by(2,0).
  covered_by(2,1).
- covered_by(2,2).



## Weak pseudocomplementation

In [117]:
goal = top_to_bot
prover = Prover9Command(goal, assumptions = ultraweak_pseudo)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18884 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 5.
% Level of proof is 2.
% Maximum clause weight is 4.000.
% Given clauses 0.


2 Not(Top) = Bot.  [goal].
19 Not(Top) = Bot.  [assumption].
20 Bot = Not(Top).  [copy(19),flip(a)].
21 Not(Top) != Bot.  [deny(2)].
22 $F.  [copy(21),rewrite([20(3)]),xx(a)].

============================== end of proof ==========================


In [118]:
goal = bot_to_top
prover = Prover9Command(goal, assumptions = ultraweak_pseudo)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18886 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 12.
% Level of proof is 3.
% Maximum clause weight is 7.000.
% Given clauses 16.


2 Not(Bot) = Top.  [goal].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
17 x = And(x,Not(Not(x))).  [assumption].
18 And(x,Not(Not(x))) = x.  [copy(17),flip(a)].
19 Not(Top) = Bot.  [assumption].
20 Bot = Not(Top).  [copy(19),flip(a)].
21 Not(Bot) != Top.  [deny(2)].
22 Not(Not(Top)) != Top.  [copy(21),rewrite([20(1)])].
4

## Pseudocomplementation

In [119]:
goal = antitone
prover = Prover9Command(goal, assumptions = pseudo)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18888 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 24.
% Level of proof is 7.
% Maximum clause weight is 11.000.
% Given clauses 58.


1 And(x,y) = Bot -> x = And(x,Not(y)).  [assumption].
2 x = And(x,y) -> Not(y) = And(Not(x),Not(y)).  [goal].
5 Or(x,y) = Or(y,x).  [assumption].
6 And(x,y) = And(y,x).  [assumption].
9 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
10 And(And(x,y),z) = And(x,And(y,z)).  [copy(9),flip(a)].
11 Or(x,And(x,y)) = x.  [assumption].
13 Or(x,Bot) = x. 

In [120]:
goal = top_to_bot
prover = Prover9Command(goal, assumptions = pseudo)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18890 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 3.
% Maximum clause weight is 7.000.
% Given clauses 16.


2 Not(Top) = Bot.  [goal].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
16 And(x,Not(x)) = Bot.  [assumption].
17 Not(Top) != Bot.  [deny(2)].
37 And(Top,x) = x.  [para(14(a,1),6(a,1)),flip(a)].
54 Not(Top) = Bot.  [para(37(a,1),16(a,1))].
55 $F.  [resolve(54,a,17,a)].

============================== end of proof ==========

In [121]:
goal = bot_to_top
prover = Prover9Command(goal, assumptions = pseudo)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18892 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 9.
% Level of proof is 3.
% Maximum clause weight is 11.000.
% Given clauses 16.


1 And(x,y) = Bot -> x = And(x,Not(y)).  [assumption].
2 Not(Bot) = Top.  [goal].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
15 And(x,y) != Bot | And(x,Not(y)) = x.  [clausify(1)].
17 Not(Bot) != Top.  [deny(2)].
37 And(Top,x) = x.  [para(14(a,1),6(a,1)),flip(a)].
51 Not(Bot) = Top.  [hyper(15,a,37,a),rewrite([37(4)])].
5

In [122]:
goal = DNI
prover = Prover9Command(goal, assumptions = pseudo)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18894 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 7.
% Level of proof is 3.
% Maximum clause weight is 11.000.
% Given clauses 12.


1 And(x,y) = Bot -> x = And(x,Not(y)).  [assumption].
2 x = And(x,Not(Not(x))).  [goal].
15 And(x,y) != Bot | And(x,Not(y)) = x.  [clausify(1)].
16 And(x,Not(x)) = Bot.  [assumption].
17 And(c1,Not(Not(c1))) != c1.  [deny(2)].
44 And(x,Not(Not(x))) = x.  [hyper(15,a,16,a)].
45 $F.  [resolve(44,a,17,a)].

============================== end of proof =

## Orthocomplementation

In [123]:
goal = top_to_bot
prover = Prover9Command(goal, assumptions = ortho)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18896 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 14.
% Level of proof is 4.
% Maximum clause weight is 7.000.
% Given clauses 18.


2 Not(Top) = Bot.  [goal].
5 Or(x,y) = Or(y,x).  [assumption].
6 And(x,y) = And(y,x).  [assumption].
13 Or(x,Bot) = x.  [assumption].
18 Or(x,Not(x)) = Top.  [assumption].
19 x = And(x,Not(Not(x))).  [assumption].
20 And(x,Not(Not(x))) = x.  [copy(19),flip(a)].
21 Not(Not(x)) = And(Not(Not(x)),x).  [assumption].
22 Not(Not(x)) = x.  [copy(21),rewrit

In [124]:
goal = bot_to_top
prover = Prover9Command(goal, assumptions = ortho)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18898 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 8.
% Level of proof is 3.
% Maximum clause weight is 7.000.
% Given clauses 17.


2 Not(Bot) = Top.  [goal].
5 Or(x,y) = Or(y,x).  [assumption].
13 Or(x,Bot) = x.  [assumption].
18 Or(x,Not(x)) = Top.  [assumption].
23 Not(Bot) != Top.  [deny(2)].
42 Or(Bot,x) = x.  [para(13(a,1),5(a,1)),flip(a)].
56 Not(Bot) = Top.  [para(42(a,1),18(a,1))].
57 $F.  [resolve(56,a,23,a)].

============================== end of proof ===============

In [125]:
goal = bot_to_neg
mb = MaceCommand(goal, assumptions = ortho + [cover_def])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 6

Bot = 0.

Top = 1.

c1 = 2.

c2 = 4.

Not(0) = 1.
Not(1) = 0.
Not(2) = 3.
Not(3) = 2.
Not(4) = 5.
Not(5) = 4.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(0,3) = 0.
And(0,4) = 0.
And(0,5) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(1,3) = 3.
And(1,4) = 4.
And(1,5) = 5.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.
And(2,3) = 0.
And(2,4) = 0.
And(2,5) = 0.
And(3,0) = 0.
And(3,1) = 3.
And(3,2) = 0.
And(3,3) = 3.
And(3,4) = 0.
And(3,5) = 0.
And(4,0) = 0.
And(4,1) = 4.
And(4,2) = 0.
And(4,3) = 0.
And(4,4) = 4.
And(4,5) = 0.
And(5,0) = 0.
And(5,1) = 5.
And(5,2) = 0.
And(5,3) = 0.
And(5,4) = 0.
And(5,5) = 5.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(0,3) = 3.
Or(0,4) = 4.
Or(0,5) = 5.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(1,3) = 1.
Or(1,4) = 1.
Or(1,5) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.
Or(2,3) = 1.
Or(2,4) = 1.
Or(2,5) = 1.
Or(3,0) = 3.
Or(3,1) = 1.
Or(3,2) = 1.
Or(3,3) = 3.
Or(3,4) = 1.
Or(3,5) = 1.
Or(4,0) = 4.
Or(

# 6. Any bounded lattice admits a weak pseudocomplementation<a id='6'></a>

In [126]:
nonbot_to_bot = read_expr('-(x=Bot) -> Not(x)=Bot')

In [127]:
goal = antitone
prover = Prover9Command(goal, assumptions = bounded_lattice + [bot_to_top, nonbot_to_bot])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18902 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 25.
% Level of proof is 8.
% Maximum clause weight is 11.000.
% Given clauses 27.


1 x != Bot -> Not(x) = Bot.  [assumption].
2 x = And(x,y) -> Not(y) = And(Not(x),Not(y)).  [goal].
5 Or(x,y) = Or(y,x).  [assumption].
6 And(x,y) = And(y,x).  [assumption].
9 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
10 And(And(x,y),z) = And(x,And(y,z)).  [copy(9),flip(a)].
11 Or(x,And(x,y)) = x.  [assumption].
13 Or(x,Bot) = x.  [assumptio

In [128]:
goal = semicomp
prover = Prover9Command(goal, assumptions = bounded_lattice + [bot_to_top, nonbot_to_bot])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18904 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 15.
% Level of proof is 4.
% Maximum clause weight is 8.000.
% Given clauses 15.


1 x != Bot -> Not(x) = Bot.  [assumption].
2 And(x,Not(x)) = Bot.  [goal].
5 Or(x,y) = Or(y,x).  [assumption].
6 And(x,y) = And(y,x).  [assumption].
11 Or(x,And(x,y)) = x.  [assumption].
13 Or(x,Bot) = x.  [assumption].
14 And(x,Top) = x.  [assumption].
15 Not(Bot) = Top.  [assumption].
16 Bot = x | Not(x) = Bot.  [clausify(1)].
17 And(c1,Not(c1)) !

In [129]:
goal = DNI
prover = Prover9Command(goal, assumptions = bounded_lattice + [bot_to_top, nonbot_to_bot])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18906 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 12.
% Level of proof is 4.
% Maximum clause weight is 7.000.
% Given clauses 13.


1 x != Bot -> Not(x) = Bot.  [assumption].
2 x = And(x,Not(Not(x))).  [goal].
4 And(x,x) = x.  [assumption].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
15 Not(Bot) = Top.  [assumption].
16 Bot = x | Not(x) = Bot.  [clausify(1)].
17 And(c1,Not(Not(c1))) != c1.  [deny(2)].
37 And(Top,x) = x.  [para(14(a,1),6(a,1)),flip(a)]

In [130]:
# If there are two nonzero elements with nonzero meet, then
# the weak pseudocomplementation above is not a pseudocomplementation

two_nonzero = read_expr('-(c=Bot) & -(d=Bot) & -(c=d) & And(c,d)=Bot')

goal = read_expr('-(0=0)')
prover = Prover9Command(goal, assumptions = bounded_lattice + pseudo + [bot_to_top, nonbot_to_bot, two_nonzero])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18908 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 20.
% Level of proof is 5.
% Maximum clause weight is 13.000.
% Given clauses 23.


1 And(x,y) = Bot -> x = And(x,Not(y)).  [assumption].
2 x != Bot -> Not(x) = Bot.  [assumption].
3 c != Bot & d != Bot & c != d & And(c,d) = Bot.  [assumption].
6 And(x,x) = x.  [assumption].
8 And(x,y) = And(y,x).  [assumption].
11 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
12 And(And(x,y),z) = And(x,And(y,z)).  [copy(11),flip(a)].
25 And(x

# 7. Any lattice $L$ with $|L|>2$ admits a precomplementation that is not a protocomplementation<a id='7'></a>



In [131]:
nontop_to_top = read_expr('-(x=Top) -> Not(x)=Top')

In [132]:
goal = antitone
prover = Prover9Command(goal, assumptions = bounded_lattice + [top_to_bot, nontop_to_top])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18912 was started by halcrow on iMac.local,
Fri Jul 15 19:52:27 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 15.
% Level of proof is 5.
% Maximum clause weight is 8.000.
% Given clauses 14.


1 x != Top -> Not(x) = Top.  [assumption].
2 x = And(x,y) -> Not(y) = And(Not(x),Not(y)).  [goal].
4 And(x,x) = x.  [assumption].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
15 Not(Top) = Bot.  [assumption].
16 Top = x | Not(x) = Top.  [clausify(1)].
17 And(c1,c2) = c1.  [deny(2)].
18 Not(c2) != And(Not(c1),Not(c2)).  [de

In [133]:
more_than_two = read_expr('-(c=Top) & -(c=Bot) ')

goal = read_expr('-(0=0)')
prover = Prover9Command(goal, assumptions = bounded_lattice + protocomp + [top_to_bot, nontop_to_top, more_than_two])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 18914 was started by halcrow on iMac.local,
Fri Jul 15 19:52:28 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 9.
% Level of proof is 3.
% Maximum clause weight is 7.000.
% Given clauses 26.


2 x != Top -> Not(x) = Top.  [assumption].
3 c != Top & c != Bot.  [assumption].
16 And(x,Top) = x.  [assumption].
27 And(x,Not(x)) = Bot.  [assumption].
30 Top = x | Not(x) = Top.  [clausify(2)].
31 c != Top.  [clausify(3)].
32 c != Bot.  [clausify(3)].
60 Top = x | Bot = x.  [para(30(b,1),27(a,1,2)),rewrite([16(4)]),flip(b)].
72 $F.  [resolve(60,b,